# Import Demand

In [616]:
import math

sentido = 'ida' # ida ou volta
vinculo = 'aluno' # aluno ou funcionario
terminal = 'butanta' # butanta ou p3
horario = '8h' # 8h e 18h

map_file_name = 'data/disciplinas/' + sentido + '_' + vinculo + '_' + terminal + '_' + horario + '.json'

acao = 'embarque' if sentido == 'volta' else 'desembarque' # dump final
contrario_acao = 'desembarque' if acao == 'embarque' else 'embarque' # filtro

hora = 1080 if horario == '18h' else 480
def intervalo_horario_am_pm(horario_pesquisa):
    if isinstance(horario_pesquisa, float) and math.isnan(horario_pesquisa):
        return False
    if horario == '18h':
        return '5:30:00 PM' <= horario_pesquisa <= '7:30:00 PM'
    else:
        return '6:30:00 AM' <= horario_pesquisa <= '8:30:00 AM'
def intervalo_horario_inicio_matrusp(horario_inicio, horario_fim):
    if horario == '18h':
        return '15:30' <= horario_inicio <= '17:30'
    else:
        return '07:30' <= horario_inicio <= '08:30'
def intervalo_horario_fim_matrusp(horario_inicio, horario_fim):
    return '17:30' <= horario_fim <= '19:00'
intervalo_horario = intervalo_horario_inicio_matrusp if sentido == 'ida' else intervalo_horario_fim_matrusp

def is_butanta(x):
    return x == 'Ponto de ônibus metrô Butantã' or x == 'Ponto de ônibus - Acesso CPTM I'
def is_p3(x):
    return x == 'Ponto de ônibus Portaria III' or x == 'Ponto de ônibus acesso Vl. Indiana'
filtra_terminal = is_butanta if terminal == 'butanta' else is_p3

def get_dados_vinculo(aulas_alunos, funcionarios_intervalo_horario):
    if vinculo == 'aluno':
        return aulas_alunos
    return funcionarios_intervalo_horario
def grad_ou_pos(vinculo):
    return vinculo == 'Aluno de graduação' or vinculo == 'Aluno de pós-graduação'
def funcionario(vinculo):
    return vinculo == 'Funcionário'
filtra_vinculo = grad_ou_pos if vinculo == 'aluno' else funcionario

dias = ['seg', 'ter', 'qua', 'qui', 'sex', 'sab', 'dom']

# Aulas MATRUSP

##### Read jsons

In [617]:
import os
import json

In [618]:
files = []
dir_path = 'data/disciplinas/json/'
for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

In [619]:
len(files)

5205

In [620]:
disciplinas = []
for file in files:
    with open(dir_path + file, 'r+') as f:
        content = json.loads(f.read())
        disciplinas.append(content)

In [621]:
disciplinas[1000]

{'unidade': 'Escola Politécnica',
 'departamento': 'Eng Telecomunicações e Controle',
 'campus': 'São Paulo',
 'codigo': 'PTC3502',
 'nome': 'Estágio Supervisionado',
 'creditos_aula': 1,
 'creditos_trabalho': 6,
 'objetivos': 'Realização de um estágio relacionado ao futuro exercício profissional, com a supervisão de um professor do curso.',
 'programa_resumido': 'Estágio em empresa, universidade ou instituição de pesquisa.',
 'turmas': [{'codigo': '2022250',
   'inicio': '15/08/2022',
   'fim': '21/12/2022',
   'tipo': 'Teórica',
   'horario': [{'dia': 'sab',
     'inicio': '09:20',
     'fim': '10:10',
     'professores': ['Cristiano Magalhaes Panazio']}],
   'vagas': {'Obrigatória': {'vagas': 35,
     'inscritos': 8,
     'pendentes': 0,
     'matriculados': 8,
     'grupos': {'EP - Engenharia Elétrica - EC3 (Telecomunicações)': {'vagas': 35,
       'inscritos': 6,
       'pendentes': 0,
       'matriculados': 6}}},
    'Extracurricular': {'vagas': 2,
     'inscritos': 0,
     'pend

##### Filtrar Institutos CUASO

In [622]:
fora_cuaso = ['Escola de Artes, Ciências e Humanidades', 'Escola de Enfermagem', 'Faculdade de Direito', 'Faculdade de Medicina',
              'Faculdade de Saúde Pública', 'Faculdade de Saúde Pública e Faculdade de Ciências Farmacêuticas', 'Escola de Enfermagem de Ribeirão Preto',
              'Museu de Zoologia']
def is_disciplina_cuaso(disciplina, fora_cuaso):
    if disciplina['campus'] != 'São Paulo':
        return False
    if disciplina['unidade'] in fora_cuaso:
        return False
    return True

In [623]:
disciplinas_cuaso = []
for disciplina in disciplinas:
    if is_disciplina_cuaso(disciplina, fora_cuaso):
        disciplinas_cuaso.append(disciplina)
len(disciplinas_cuaso)

2352

In [624]:
unidades_matrusp = {}
for disciplina in disciplinas_cuaso:
    unidades_matrusp[disciplina['unidade']] = 1
unidades_matrusp

{'Instituto Oceanográfico': 1,
 'Escola Politécnica': 1,
 'Escola de Comunicações e Artes': 1,
 'Instituto de Matemática e Estatística': 1,
 'Faculdade de Filosofia, Letras e Ciências Humanas': 1,
 'Instituto de Química': 1,
 'Instituto de Ciências Biomédicas': 1,
 'Instituto de Geociências': 1,
 'Instituto de Biociências': 1,
 'Instituto de Psicologia': 1,
 'Faculdade de Medicina Veterinária e Zootecnia': 1,
 'Faculdade de Ciências Farmacêuticas': 1,
 'Escola de Educação Física e Esporte': 1,
 'Faculdade de Educação': 1,
 'Instituto de Física': 1,
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 1,
 'Faculdade de Odontologia': 1,
 'Faculdade de Arquitetura e Urbanismo': 1,
 'Instituto de Relações Internacionais': 1,
 'Faculdade de Economia, Administração, Contabilidade e Atuária': 1,
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 1,
 'Instituto de Estudos Brasileiros': 1,
 'Museu de Arqueologia e Etnologia': 1

##### Gerar Lista de Aulas

In [625]:
aulas = []
for i, disciplina in enumerate(disciplinas_cuaso):
    for turma in disciplina['turmas']:
        total_matriculados = 0
        for k, vaga in turma['vagas'].items():
            total_matriculados += vaga['matriculados']
            
        if turma['horario'] != None:
            for horarios in turma['horario']:
                aula = {}
                aula['indice_disc'] = i
                aula['codigo'] = disciplina['codigo']
                aula['codigo_turma'] = turma['codigo']
                aula['unidade'] = disciplina['unidade']
                aula['dia'] = horarios['dia']
                aula['horario_inicio'] = horarios['inicio']
                aula['horario_fim'] = horarios['fim']
                aula['total'] = total_matriculados
                aulas.append(aula)

##### Filtrar Aulas por Intervalo Horário

In [626]:
aulas_intervalo_horario = []
for aula in aulas:
    if intervalo_horario(aula['horario_inicio'], aula['horario_fim']):
        aulas_intervalo_horario.append(aula)

# Pesquisa Parte 1

##### Ler Pesquisa

In [627]:
import pandas as pd
import datetime as dt

In [628]:
pesquisa_perguntas = pd.read_csv('./data/disciplinas/forms/resposta_pesq.csv', delimiter=';')
pesquisa_perguntas['timestamp'] = list(map(lambda x: dt.datetime.strptime(x, '%m/%d/%Y %H:%M:%S'), pesquisa_perguntas['Carimbo de data/hora']))
pesquisa_perguntas = pesquisa_perguntas[pesquisa_perguntas['timestamp'] >= dt.datetime(2022, 10, 7, 0, 0, 0)]
len(pesquisa_perguntas)

4395

In [629]:
pesquisa = pd.DataFrame()
pesquisa['vinculo'] = pesquisa_perguntas['Qual o seu vínculo com a USP?']
pesquisa['instituto'] = pesquisa_perguntas['A qual instituto você está associado?']


# ida
pesquisa['utiliza_circular_ida'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para entrar na USP?'] >= 2
pesquisa['linhas_ida'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?']
pesquisa['ponto_embarque_ida'] = pesquisa_perguntas['Por qual ponto você normalmente embarca?']
pesquisa['ponto_desembarque_ida'] = pesquisa_perguntas['Em qual ponto de ônibus você normalmente desembarca?']
pesquisa['horario_ida'] = pesquisa_perguntas['Que horas você normalmente chega na USP?']


# volta
pesquisa['utiliza_circular_volta'] = pesquisa_perguntas['Quantas vezes por semana você utiliza o circular para sair da USP?'] >= 2
pesquisa['linhas_volta'] = pesquisa_perguntas['Qual das linhas abaixo você utiliza neste trajeto?.1']
pesquisa['ponto_embarque_volta'] = pesquisa_perguntas['Em qual o ponto de ônibus você embarca para sair da USP?']
pesquisa['ponto_desembarque_volta'] = pesquisa_perguntas['Por qual ponto você normalmente desembarca?']
pesquisa['horario_volta'] = pesquisa_perguntas['Que horas você normalmente chega no ponto de ônibus para sair da USP?']

pesquisa.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculadae de Arquitetura e Urbanismo (FAU),True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística (IME),True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Huma...",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações internacionais (IRI),False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


##### Verificações

In [630]:
# Verifica quantas pessoas que saem do P3 e vão para o Fau II
pesquisa_emb_des_ida = pesquisa[['ponto_embarque_ida', 'ponto_desembarque_ida']]
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_embarque_ida'] == 'P3']
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_desembarque_ida'] == 'FAU II']
len(pesquisa_emb_des_ida)

29

In [631]:
# Verifica quantas pessoas que saem do P3 e vão para o Fau I
pesquisa_emb_des_ida = pesquisa[['ponto_embarque_ida', 'ponto_desembarque_ida']]
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_embarque_ida'] == 'P3']
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_desembarque_ida'] == 'FAU I']
len(pesquisa_emb_des_ida)

1

In [632]:
# Verifica quantas pessoas que saem do P3 e vão para o P3
pesquisa_emb_des_ida = pesquisa[['ponto_embarque_ida', 'ponto_desembarque_ida']]
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_embarque_ida'] == 'P3']
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_desembarque_ida'] == 'P3']
len(pesquisa_emb_des_ida)

17

In [633]:
# Verifica quantas pessoas que saem do P3 e vão para o P3
pesquisa_emb_des_ida = pesquisa[['ponto_embarque_ida', 'ponto_desembarque_ida']]
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_embarque_ida'] == 'Estação Butantã']
pesquisa_emb_des_ida = pesquisa_emb_des_ida[pesquisa_emb_des_ida['ponto_desembarque_ida'] == 'Estação Butantã']
len(pesquisa_emb_des_ida)

106

In [634]:
# Verifica a situação de números iguais na poli mecânica
pesquisa_filtered_mec = pesquisa[['horario_ida', 'vinculo', 'ponto_embarque_ida', 'ponto_desembarque_ida', 'instituto', 'linhas_ida', 'utiliza_circular_ida']]
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['ponto_embarque_ida'].isin(['P3', 'Portão da Vila Indiana'])]
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['instituto'] == 'Escola Politécnica (Poli)']
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['ponto_desembarque_ida'] == 'Poli Mecânica']
pesquisa_filtered_mec

,horario_ida,vinculo,ponto_embarque_ida,ponto_desembarque_ida,instituto,linhas_ida,utiliza_circular_ida
2330,1:40:00 PM,Aluno de graduação,P3,Poli Mecânica,Escola Politécnica (Poli),"8012, 8022",True
3341,8:15:00 AM,Aluno de pós-graduação,Portão da Vila Indiana,Poli Mecânica,Escola Politécnica (Poli),"8012, 8022, 8032",True


In [635]:
# Verifica o aumento de cerca de 1000 alunos na ida do butantã de manhã
pesquisa_filtered_mec = pesquisa[['horario_ida', 'vinculo', 'ponto_embarque_ida', 'utiliza_circular_ida']]
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['ponto_embarque_ida'].isin(['Estação Butantã', 'PTREM'])]
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['vinculo'].isin(['Aluno de graduação', 'Aluno de pós-graduação'])]
pesquisa_filtered_mec = pesquisa_filtered_mec[pesquisa_filtered_mec['utiliza_circular_ida']]
pesquisa_filtered_mec_6_as_8 = pesquisa_filtered_mec['6:30:00 AM' <= pesquisa_filtered_mec['horario_ida']]
pesquisa_filtered_mec_6_as_8 = pesquisa_filtered_mec_6_as_8[pesquisa_filtered_mec_6_as_8['horario_ida'] <= '8:30:00 AM']
pesquisa_filtered_mec_7_as_8 = pesquisa_filtered_mec['7:30:00 AM' <= pesquisa_filtered_mec['horario_ida']]
pesquisa_filtered_mec_7_as_8 = pesquisa_filtered_mec_7_as_8[pesquisa_filtered_mec_7_as_8['horario_ida'] <= '8:30:00 AM']
aumento = (len(pesquisa_filtered_mec_6_as_8)/len(pesquisa_filtered_mec_7_as_8) - 1)*100
print('Aumento de ' + str(aumento) + '%')

Aumento de 51.387461459403916%


In [636]:
pesquisa['instituto'].unique()

array(['Instituto de Matemática e Estatística (IME)',
       'Faculadae de Arquitetura e Urbanismo (FAU)',
       'Faculdade de Filosofia, Letras e Ciências Humanas (FFLCH)',
       'Instituto de Relações internacionais (IRI)',
       'Faculdade de Ciências Farmacêuticas (FCF)',
       'Escola Politécnica (Poli)',
       'Faculdade de Economia, Administração e Contabilidade (FEA)',
       'Instituto de Química (IQ)',
       'Escola de Artes, Ciências e Humanidades (EACH)',
       'Instituto de Física (IF)', 'Faculdade de Medicina (FM)',
       'Instituto Oceanográfico (IO)',
       'Escola de Comunicações e Artes (ECA)',
       'Instituto de Pesquisas Energéticas e Nucleares (IPEN)',
       'Instituto de Biociências (IB)', 'Faculdade de Educação (FE)',
       'Instituto de Ciências Biomédicas (ICB)',
       'Faculdade de Odontologia (FO)',
       'Escola de Educação Física e Esporte (EEFE)', 'PRG',
       'Instituo de Astronomia, Geofísica e Ciências Atmosféricas (IAG)',
       'Instit

##### Filtrar Vínculo

In [637]:
pesquisa[['vinculo']].value_counts()

vinculo                                                        
Aluno de graduação                                                 2880
Aluno de pós-graduação                                              752
Funcionário                                                         644
Docente                                                              36
docente                                                              22
Professor                                                            21
professor                                                             8
Ex-aluno                                                              5
Professor                                                             5
Docente                                                               4
Professor titular                                                     1
aprimoramento                                                         1
Pós-doutorando                                                        1


In [638]:
pesquisa[['vinculo', 'utiliza_circular_' + sentido]].value_counts()

vinculo                                                          utiliza_circular_ida
Aluno de graduação                                               True                    2532
Aluno de pós-graduação                                           True                     575
Funcionário                                                      True                     369
Aluno de graduação                                               False                    348
Funcionário                                                      False                    275
Aluno de pós-graduação                                           False                    177
Docente                                                          True                      28
docente                                                          False                     17
Professor                                                        True                      11
                                                                 Fal

In [639]:
pesquisa_vinculo = pesquisa[list(map(filtra_vinculo, pesquisa['vinculo']))]
pesquisa_vinculo[['vinculo']].value_counts()

vinculo               
Aluno de graduação        2880
Aluno de pós-graduação     752
dtype: int64

##### Normaliza instituto com unidade do matrusp ou anuário

In [640]:
def retira_sigla(x):
    index = x.find('(')
    
    if index < 0:
        return x
    
    return x[:index - 1]

def filtra_instituto_pesq(x):
    x = retira_sigla(x)
    
    if x in ['ECA e MAC USP', 'Escola de Artes, Ciências e Humanidades']: 
        return 'Escola de Comunicações e Artes'
    elif x == 'FFLCH':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x in ['IPEN', 'Instituto de Pesquisas Energéticas e Necleares', 'IPEN ', 'ipen', 'Ipen', 'Instituto de pesquisas energéticas e nucleares', 'Instituto de Pesquisa Energéticas e Nucleares', 'Instituto de Pesquisas Energéticas e Nucleares - IPEN', 'Instituto de pesquisas energéticas e nucleares ', 'Instituto de pesquisas energéticas e nucleares -IPEN', 'Instituo de Pesquisas Energéticas e Nucleares']:
        return 'Instituto de Pesquisas Energéticas e Nucleares'
    elif x in ['MAC', 'MAC USP']:
        return 'Museu de Arte Contemporânea'
    elif x == 'Faculadae de Arquitetura e Urbanismo':
        return 'Faculdade de Arquitetura e Urbanismo'
    elif x in ['IAG ', 'IAG', 'Instituo de Astronomia, Geofísica e Ciências Atmosféricas']:
        return 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas'
    elif x == 'Faculdade de Economia, Administração e Contabilidade':
        return 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    elif x in ['Escola de Aplicação da Faculdade de Educação (EA-FEUSP)']:
        return 'Escola de Educação Física e Esporte'
    elif x == 'Instituto de Relações internacionais':
        return 'Instituto de Relações Internacionais'
    elif x in ['Museu de arqueologia e etnologia ']:
        return 'Museu de Arqueologia e Etnologia'
    elif x in ['Instituto de Energia e Ambientes']:
        return 'Instituto de Energia e Ambiente'
    elif x in ['Hospital universitário da usp', 'Hospital universitário ', 'Hospital HU', 'Hosp. Universitario', 'HU-USP', 'Hospital Universitário da USP ', 'HOSPITAL UNIVERSITÁRIO', 'HOSPITAL UNIVERSITÁRIO - HU', 'Hospital Universitário da USP', 'HU USP', 'hospital universitário', 'HU', 'Hospital Universitário HU', 'Hu', 'HOSPITAL UNIVERSITARIO', 'Hospital universitário', 'Hospital  Universitário ', 'hu', 'HU ', 'Hospital Universitário ']:
        return 'Hospital Universitário'
    elif x in ['SAS', 'sas', 'Superintendência de Comunicação Social', 'Prip', 'edusp', 'pusp c', 'PUSP-C','pusp-c', 'Prédio da Reitoria', 'Prédio da Administração Central-SEF', 'Prefeitura do Campus USP da Capital', 'SUB REITORIA', 'Reitoria.', 'Prip usp', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP', 'pro reitoria', 'Pro reitoria de cultura e extensão ', 'PRIP/Reitoria', 'Edusp', 'PUSP C', 'PUS-C', 'Pusp', 'Prefeitura do Campos', 'Prefeitura ', 'Reitoria da USP', 'Editora da Universidade de São Paulo - Edusp', 'Pusp-c', 'PRG', 'Prg ', 'Pró-Reitoria de Graduação', 'Pró-Reitoria de graduação', 'Pró-reitoria de graduacao', 'Reitoria', 'Reitoria usp', 'REITORIA', 'Pró-Reitoria de Inclusão e Pertencimento - PRIP-USP', 'Pró-Reitoria de Inclusão e Pertencimento', 'Reitoria da Universidade de São Paulo', 'Reitoria - Administração Central', 'Reitoria ', 'rusp', 'RUSP', 'Editora da USP']:
        return 'Reitoria Administração Central'
    elif x in ['Museu de Zoologia', 'Museu de Zoologia da USP']:
        return 'Faculdade de Medicina Veterinária e Zootecnia'
    elif x in ['Superintendencia de Tecnologia da Informação', 'superintendencia do espaço fisico', 'Superintendência de Segurança - SPPU', 'SUperintendencia de Tecnologia e INformação', 'STI - CeTI-SP']:
        return 'Superintendência da Tecnologia da Informação'
    elif x in ['Cepeusp ', 'Cepeusp', 'Centro de Práticas Esportivas da USP - CEPEUSP', 'CEPEUSP']:
        return 'Centro de Práticas Esportivas'
    elif x in ['Curso de Ciências Moleculares', 'Prceu ', 'PRCEU', 'Clinica Odontológica da SAU no CRUSP', 'PRIP', 'STI', 'Superintendência de Tecnologia da Informação - STI', 'CEPEUSP ', 'SEF', 'Coseas', 'Superintendência do Espaço Físico', 'BBM-USP', 'CeTISP', 'ABCD', 'Restaurante universitario central', 'Restaurante Central ', 'Aucani', 'Centro de Saúde Escola Samuel Barsley Pessoa ', 'Osusp', 'SCS', 'Museu Paulista', 'Museu de Arqueologia e Etnologia ', 'CSCRH-RP-01', 'Instituto de Medicina Tropical de São Paulo', 'Rádio USP ', 'Perceu', 'USP Centro de Tecnologia de Informação - CETISP']:
        None
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(filtra_instituto_pesq, pesquisa_vinculo['instituto']))

In [641]:
# Filtra os Nones

pesquisa_vinculo = pesquisa_vinculo2
pesquisa_vinculo = pesquisa_vinculo[pesquisa_vinculo['instituto'].notna()]
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística',
       'Faculdade de Arquitetura e Urbanismo',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Instituto de Relações Internacionais',
       'Faculdade de Ciências Farmacêuticas', 'Escola Politécnica',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto de Química', 'Escola de Comunicações e Artes',
       'Instituto de Física', 'Faculdade de Medicina',
       'Instituto Oceanográfico',
       'Instituto de Pesquisas Energéticas e Nucleares',
       'Instituto de Biociências', 'Faculdade de Educação',
       'Instituto de Ciências Biomédicas', 'Faculdade de Odontologia',
       'Escola de Educação Física e Esporte',
       'Reitoria Administração Central',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Geociências',
       'Faculdade de Medicina Veterinária e Zootecnia',
       'Faculdade de Saúde Pública', 'Escola de Enfermagem',
       '

##### Mapeia Institutos Fora CUASO

In [642]:
def mapeia_cursos_de_fora(x):
    if x in ['Faculdade de Medicina', 'Escola de Enfermagem', 'Faculdade de Saúde Pública']:
        return 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    elif x == 'Faculdade de Direito':
        return 'Faculdade de Filosofia, Letras e Ciências Humanas'
    elif x == 'MAC':
        return 'Museu de Arte Contemporânea'
    else:
        return x
    
pesquisa_vinculo2 = pesquisa_vinculo.copy()
pesquisa_vinculo2['instituto'] = list(map(mapeia_cursos_de_fora, pesquisa_vinculo['instituto']))
pesquisa_vinculo = pesquisa_vinculo2

In [643]:
pesquisa_vinculo['instituto'].unique()

array(['Instituto de Matemática e Estatística',
       'Faculdade de Arquitetura e Urbanismo',
       'Faculdade de Filosofia, Letras e Ciências Humanas',
       'Instituto de Relações Internacionais',
       'Faculdade de Ciências Farmacêuticas', 'Escola Politécnica',
       'Faculdade de Economia, Administração, Contabilidade e Atuária',
       'Instituto de Química', 'Escola de Comunicações e Artes',
       'Instituto de Física',
       'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
       'Instituto Oceanográfico',
       'Instituto de Pesquisas Energéticas e Nucleares',
       'Instituto de Biociências', 'Faculdade de Educação',
       'Instituto de Ciências Biomédicas', 'Faculdade de Odontologia',
       'Escola de Educação Física e Esporte',
       'Reitoria Administração Central',
       'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
       'Instituto de Geociências',
       'Faculdade de Medicina Veter

##### Filtrar respostas pesquisa pelo Intervalo Horário

In [644]:
pesquisa_vinculo.head()

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta
136,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM
137,Aluno de graduação,Faculdade de Arquitetura e Urbanismo,True,"8012, 8032",Estação Butantã,FEA,1:10:00 PM,True,"8012, 8022",FAU II,Estação Butantã,10:30:00 PM
138,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM
140,Aluno de graduação,Instituto de Relações Internacionais,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN


In [645]:
pesquisa_vinculo['no_horario'] = list(map(intervalo_horario_am_pm, pesquisa_vinculo['horario_' + sentido]))
pesquisa_vinculo_intervalo = pesquisa_vinculo[pesquisa_vinculo['no_horario']][pesquisa_vinculo.columns]
pesquisa_vinculo_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario
136,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8022",P3,FAU II,7:54:00 AM,True,"8012, 8022",FAU II,Portão da Vila Indiana,7:10:00 PM,True
138,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8032",Estação Butantã,FEA,7:15:00 PM,True,8022,Rua do Matão 0,Estação Butantã,10:40:00 PM,True
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8012,PTREM,Geografia/História,7:25:00 PM,True,8012,Geografia/História,PTREM,9:40:00 PM,True
141,Aluno de graduação,Faculdade de Ciências Farmacêuticas,True,8022,P3,Farmácia e Química,6:55:00 PM,True,8022,Butantan,Estação Butantã,10:00:00 PM,True
142,Aluno de graduação,Escola Politécnica,True,"8022, 8032",Estação Butantã,Poli Metalurgia,7:40:00 AM,True,"8012, 8032",Poli Mecânica,Estação Butantã,3:00:00 PM,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4525,Aluno de graduação,Escola Politécnica,True,"8012, 8022, 8032",Estação Butantã,Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Poli Civil,Estação Butantã,5:00:00 PM,True
4526,Aluno de graduação,Instituto de Biociências,True,8022,Estação Butantã,Rua do Lago,7:00:00 AM,True,"8012, 8022",Biociências,Estação Butantã,7:15:00 PM,True
4527,Aluno de graduação,Instituto de Biociências,True,"8022, 8032",Estação Butantã,Biociências,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Estação Butantã,10:15:00 PM,True
4528,Aluno de graduação,Escola de Comunicações e Artes,True,"8012, 8032",Estação Butantã,CRUSP (Praça da Reitoria),8:10:00 AM,True,8012,ECA,Estação Butantã,6:00:00 PM,True


# Funcionários

##### Ler os dados do anuário

In [646]:
with open('data/funcionarios_anuario.json', 'r+') as f:
    str_funcionarios_anuario = f.read()
    funcionarios_anuario = json.loads(str_funcionarios_anuario)
funcionarios_anuario[0]

{'unidade': 'Faculdade de Arquitetura e Urbanismo', 'funcionarios': 129}

##### Filtrar Horário baseado na pesquisa

In [647]:
no_horario_por_instituto = pesquisa_vinculo[['instituto', 'no_horario']].groupby('instituto').agg('mean').reset_index()
no_horario_por_instituto[no_horario_por_instituto['instituto'] == 'Instituto de Matemática e Estatística']

,instituto,no_horario
21,Instituto de Matemática e Estatística,0.677249


In [648]:
funcionarios_intervalo_horario = []
if vinculo == 'funcionario':
    for funcionario_anuario in funcionarios_anuario:
        unidade = funcionario_anuario['unidade']
        porc_no_horario = no_horario_por_instituto[no_horario_por_instituto['instituto'] == unidade]['no_horario'].iloc[0]
        for dia in dias:
            funcionarios_intervalo_horario.append({
                'unidade': unidade,
                'dia': dia,
                'total': funcionario_anuario['funcionarios'] * porc_no_horario,
            })
    funcionarios_intervalo_horario[0]

# Pesquisa Parte 2

In [649]:
pesquisa_vinculo_intervalo['ponto_embarque_ida'].unique()

array(['P3', 'Estação Butantã', 'PTREM',
       'R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)',
       'Portaria São Remo', 'P1', 'Portão da Vila Indiana',
       'Biblioteca Brasiliana', 'P2',
       'R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)',
       'CRUSP (Praça da Reitoria)'], dtype=object)

##### Mapeia Pontos Pesquisa para ficar igual ao da API

In [650]:
import requests
pontos = requests.get('https://uspdigital.usp.br/mobile/servicos/mapa/locais?campus=cidade-universitaria&categorias=transportes')
pontos = json.loads(pontos.text)
pontos[10]

{'categoria': 'transportes',
 'campus': 'cidade-universitaria',
 'id': '1811',
 'titulo': 'Ponto de ônibus ECA',
 'latitude': '-23.557806338669685',
 'longitude': '-46.72689100000002',
 'endereco': 'Av. Prof. Lúcio Martins Rodrigues',
 'imagem': 'http://www.usp.br/mapas/wp-content/uploads/10042014giroculturalfotomarcossantos008.jpg',
 'url': '',
 'descricao': 'Atende as linhas  701U-10, 702U-10, 7725-10, 8012-10'}

In [651]:
nomes_pontos_req = []
for ponto in pontos:
    if ponto['campus'] == 'cidade-universitaria':
        nomes_pontos_req.append(ponto['titulo'])
 
nomes_pontos_pesq = []
for ponto in pesquisa_vinculo_intervalo['ponto_' + acao + '_' + sentido].unique():
    nomes_pontos_pesq.append(ponto)

print(nomes_pontos_req)
print(nomes_pontos_pesq)

['Ponto de ônibus metrô Butantã', 'Ponto de ônibus Afrânio Peixoto', 'Ponto de ônibus Poli Metalúrgica', 'Terminal de ônibus USP', 'Ponto de ônibus Poli Mecânica', 'Ponto de ônibus portaria II', 'Ponto de ônibus Hidráulica', 'Ponto de Psicologia I', 'Ponto de ônibus Barracões', 'Ponto de ônibus ECA', 'Ponto de ônibus Praça do Relógio', 'Ponto de ônibus - Praça do Relógio', 'Ponto de ônibus Psicologia II', 'Ponto de ônibus - Acesso CPTM II', 'Ponto de ônibus - Acesso CPTM I', 'Ponto de ônibus Escola de Educação Física II', 'Ponto de ônibus Escola de Educação Física I', 'Ponto de ônibus Academia de Polícia', 'Ponto de ônibus Paços das Artes', 'Ponto de ônibus Educação', 'Ponto de ônibus CRUSP', 'Ponto de ônibus Cultura Japonesa', 'Ponto de ônibus Biblioteca Brasiliana', 'Ponto de ônibus Letras', 'Ponto de ônibus Geociência', 'Ponto de ônibus Reitoria/Bancos', 'Ponto de ônibus FEA', 'Ponto de ônibus FAU II', 'Ponto de ônibus Poli Biênio', 'Ponto de ônibus Poli Eletrotécnica', 'Ponto de ôn

In [652]:
def iguais_com_prefixo(x):
    aux_nomes_pontos_pesq = ['Ponto de ônibus ' + x for x in nomes_pontos_pesq]
    if 'Ponto de ônibus ' + x in [item for item in aux_nomes_pontos_pesq if item in list(set(nomes_pontos_req) & set(aux_nomes_pontos_pesq))]:
        return 'Ponto de ônibus ' + x
    else:
        return x
 
def pesquisa_to_pontos_req(x):
    if x in ["Geografia/História", "História e Geografia", "Químicas"]:
        return "Ponto e ônibus História e Geografia"
    elif x  == "Poli Metalurgia":
        return "Ponto de ônibus Poli Metalúrgica"
    elif x == "P3":
        return "Ponto de ônibus Portaria III"
    elif x == "Farmácia e Química":
        return "Ponto de ônibus Biblioteca Farmácia e Química"
    elif x in ["CRUSP (Praça da Reitoria)", "R. Prof. Mello Moraes, 1473 (localizado entre a CRUSP e a Raia, sentindo Metrô Butantã)", "R. Prof. Mello Moraes, 900 (localizado entre a CRUSP e a Raia, sentindo P3)"]:
        return "Ponto de ônibus CRUSP"
    elif x == "Biomédicas":
        return "Ponto de ônibus Biomédicas III"
    elif x in ["Prefeitura/Física", "Ponto de ônibus Ponto Clube dos funcionários"]:
        return "Ponto Clube dos funcionários"
    elif x == "Educação Física":
        return "Ponto de ônibus Escola de Educação Física I"
    elif x in ["Estação Butantã", "Terminal Cidade Universitária"]:
        return "Ponto de ônibus metrô Butantã"
    elif x == "Poli-Eletrotécnia":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x == "Geociências":
        return "Ponto de ônibus Geociências I"
    elif x in ["icb ", "Biociências", "ICB 1 (Av. Prof. Lineu Prestes, 1524)"]:
        return "Ponto de ônibus  Parada Matemática"
    elif x in ["FE", "Avenida da Universidade", "Faculdade de Educação ", "Educação / CEPEUSP", " FE", "Prédio da FE", "Educação ", "Educação", "Faculdade de Educação"]:
        return "Ponto de ônibus Educação"
    elif x in ["Rua do matão frente da FAU", "Instituto Oceanografico", "Oceanográfico, em frent ao da FAU II", "Rua do Matão, 0 (em frente a FAU)", "Estacionamento FAU", "R. do Matão, frente à FAU", "R. do Matão 878", "R. do Matão, 0", "Oceanográfico", "Oceanografia", "Rua do Matao ", "Matão, Oceanografico", "Rua do Matão, 0 - FAU", "IO USP", "IO", "Rua do matao, 0", "Rua do Matão 0 (FAU)", "Rua do Matão (na frente da FAU)", "Intuito Oceanografico", "R. do Matão", "Oceanografia ", "Oceanógrafico/Matão", "Rua do Matão, 0", "R. Do matão ",  "Rua do Matão", "Ponto em frente à FAU na Rua do Matão"]:
        return "Ponto de ônibus FAU I"
    elif x in ["depois da biblioteca da química ", "Rua do Lago", "Lineu Prestes entre a biblioteca da quimica e o icb", "Ponto do Lago", "Av. Prof. Lineu Prestes, 0"]:
        return "Ponto de ônibus rua do Lago"
    elif x == "Psicologia I":
        return "Ponto de ônibus Psicologia II"
    elif x in ["Cepam",  "Biomédicas, Biologia e Quimica"]:
        return "Ponto de ônibus CEPAM"
    elif x in ["Ipen", "FO ou Ipen", "Marinha/ Odontologia"]:
        return "Ponto de ônibus IPEN"
    elif x in ["Acesso Vila Indiana", "Portão da Vila Indiana"]:
        return "Ponto de ônibus acesso Vl. Indiana"
    elif x == "PTREM":
        return "Ponto de ônibus - Acesso CPTM I"
    elif x == "IPT":
        return "Ponto de ônibus Cocesp I"
    elif x in ["Centro De Difusão Internacional", "Poli Hidráulica"]:
        return "Ponto de ônibus Hidráulica"
    elif x == "Avenida Professor Luciano Gualberto (depois da Parada I.P.T, no 8022)":
        return "Ponto de ônibus Poli Eletrotécnica"
    elif x in ["P1", "Instituto Butantan ", "Academia de polícia"]:
        return "Ponto de ônibus Academia de Polícia"
    elif x == "P2":
        return "Ponto de ônibus portaria II"
    elif x in ["FAU ", "FEA ou FAU II", "As vezes desembarco na FAU II ou na FEA"]:
        return "Ponto de ônibus FAU II"
    elif x == "Portaria São Remo":
        return "Ponto de ônibus acesso Rio Pequeno"
    elif x == "If e Ipen":
        return "Ponto de ônibus Física"
    elif x == "Casa de Cultura Japonesa, ETEC CEPAM":
        return "Ponto de ônibus Cultura Japonesa"
    elif x == "IEB":
        return "Ponto de ônibus Biblioteca Brasiliana"
    elif x in ["IAG (ou, às vezes, Física)", "IAG ou Física", "IAG (sentido Metrô Butantã)"]:
        return "Ponto de ônibus IAG"
    elif x == "Prof Almeida Prado":
        return "Terminal de ônibus USP"
    elif x == "HU":
        return "Ponto de ônibus da COPESP"
    elif x == "pço das artes":
        return "Ponto de ônibus Paços das Artes"
    elif x == "Depende do dia":
        return None
    else:
        return x

In [653]:
str_pontos = ['ponto_desembarque_ida', 'ponto_embarque_ida', 'ponto_desembarque_volta', 'ponto_embarque_volta']
for str_ponto in str_pontos:
    pesquisa_vinculo_intervalo[str_ponto] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_vinculo_intervalo[str_ponto]))
    pesquisa_vinculo_intervalo = pesquisa_vinculo_intervalo[pesquisa_vinculo_intervalo[str_ponto].notna()]

    for ponto in pesquisa_vinculo_intervalo[str_ponto].unique():
        if ponto not in nomes_pontos_req:
            print(ponto)

Ponto da Educação
Ponto barracões 
Crisp próximo ao bandejão 
Lineu, perto da rua do matao 
Feusp
FE-USP ou Bandejão central


/var/folders/wt/sljrl31144s8xk7szzq6n62h0000gn/T/com.apple.shortcuts.mac-helper/ipykernel_67004/2848880336.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pesquisa_vinculo_intervalo[str_ponto] = list(map(lambda x: pesquisa_to_pontos_req(iguais_com_prefixo(x)), pesquisa_vinculo_intervalo[str_ponto]))


Rua do Matão 0
icb
Oceanográfico (Matão, 0)
Oceanográfico 
Paço das Artes
Nenhum. Não saio da USP com o Circular
Existem problemas metodológicos aqui, visto que não embarco para voltar da usp, apenas para ir. Não consigo especificar isso em momento algum. Também, na página sobre como vou para a usp, consta apenas uma opção de horário. Eu vou para a usp em três horários diferentes. Sobre a próxima pergunta, em que ponto desembarco, eu não uso o busp para sair da usp. Devo responder o que?
Paço das artes/FEUSP
Matão 0
IQ
Lineu Prestes, 338.
CEPE
Pontos da rua do Matão perto do IO
Na frente da fflch do outro lado da rua
Oceanografico
InovaUSP
Entre o bandejão e o CEPE
N embarco 
Paço de Artes
Rua do Matão, do lado da FAU
Rua do Matão (do lado do IO) 
Os na Rua do Lago perto da FAU
IME 
FEA ou FÍSICA 
Feusp
Do outro lado do ponto da Educação (não sei o nome)
R. Do Matão, 0 // a maioria das pessoas da fau e do ime prefere pegar o circular em dos dois pontos que fica na rua do matão pois pas

##### Calcular Porcentagem Terminal por Instituto

In [654]:
pesquisa_vinculo_intervalo['utiliza_' + terminal] = list(map(filtra_terminal, pesquisa_vinculo_intervalo['ponto_' + contrario_acao + '_' + sentido]))
pesquisa_vinculo_intervalo['utiliza_' + terminal].mean()

0.8012326656394453

In [655]:
porc_terminal_por_instituto = pesquisa_vinculo_intervalo[['instituto','utiliza_' + terminal]].groupby('instituto').agg('mean').reset_index()
porc_terminal_por_instituto

,instituto,utiliza_butanta
0,Escola Politécnica,0.785965
1,Escola de Aplicação da Faculdade de Educação,0.000000
2,Escola de Comunicações e Artes,0.780488
3,Escola de Educação Física e Esporte,0.888889
4,Faculdade de Arquitetura e Urbanismo,0.849315
5,Faculdade de Ciências Farmacêuticas,0.833333
6,"Faculdade de Economia, Administração, Contabil...",0.852071
7,Faculdade de Educação,0.888889
8,"Faculdade de Filosofia, Letras e Ciências Humanas",0.842105
9,Faculdade de Medicina Veterinária e Zootecnia,0.728814


In [656]:
def porc_utiliza_terminal(instituto):
    return porc_terminal_por_instituto[porc_terminal_por_instituto['instituto'] == instituto]['utiliza_' + terminal].iloc[0]

##### Filtrar Terminal

In [657]:
pesquisa_vinculo_intervalo

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario,utiliza_butanta
136,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8022",Ponto de ônibus Portaria III,Ponto de ônibus FAU II,7:54:00 AM,True,"8012, 8022",Ponto de ônibus FAU II,Ponto de ônibus acesso Vl. Indiana,7:10:00 PM,True,False
138,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus FEA,7:15:00 PM,True,8022,Rua do Matão 0,Ponto de ônibus metrô Butantã,10:40:00 PM,True,True
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8012,Ponto de ônibus - Acesso CPTM I,Ponto e ônibus História e Geografia,7:25:00 PM,True,8012,Ponto e ônibus História e Geografia,Ponto de ônibus - Acesso CPTM I,9:40:00 PM,True,True
141,Aluno de graduação,Faculdade de Ciências Farmacêuticas,True,8022,Ponto de ônibus Portaria III,Ponto de ônibus Biblioteca Farmácia e Química,6:55:00 PM,True,8022,Ponto de ônibus Butantan,Ponto de ônibus metrô Butantã,10:00:00 PM,True,False
142,Aluno de graduação,Escola Politécnica,True,"8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Metalúrgica,7:40:00 AM,True,"8012, 8032",Ponto de ônibus Poli Mecânica,Ponto de ônibus metrô Butantã,3:00:00 PM,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4525,Aluno de graduação,Escola Politécnica,True,"8012, 8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Ponto de ônibus Poli Civil,Ponto de ônibus metrô Butantã,5:00:00 PM,True,True
4526,Aluno de graduação,Instituto de Biociências,True,8022,Ponto de ônibus metrô Butantã,Ponto de ônibus rua do Lago,7:00:00 AM,True,"8012, 8022",Ponto de ônibus Parada Matemática,Ponto de ônibus metrô Butantã,7:15:00 PM,True,True
4527,Aluno de graduação,Instituto de Biociências,True,"8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Parada Matemática,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Ponto de ônibus metrô Butantã,10:15:00 PM,True,True
4528,Aluno de graduação,Escola de Comunicações e Artes,True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus CRUSP,8:10:00 AM,True,8012,Ponto de ônibus ECA,Ponto de ônibus metrô Butantã,6:00:00 PM,True,True


In [658]:
if vinculo == 'funcionario':
    pesquisa_vinculo_intervalo = pesquisa_vinculo_intervalo[pesquisa_vinculo_intervalo['utiliza_circular_' + sentido]]
pesquisa_vinculo_intervalo_terminal = pesquisa_vinculo_intervalo.copy()[list(map(filtra_terminal, pesquisa_vinculo_intervalo['ponto_' + contrario_acao + '_' + sentido]))]
pesquisa_vinculo_intervalo_terminal

,vinculo,instituto,utiliza_circular_ida,linhas_ida,ponto_embarque_ida,ponto_desembarque_ida,horario_ida,utiliza_circular_volta,linhas_volta,ponto_embarque_volta,ponto_desembarque_volta,horario_volta,no_horario,utiliza_butanta
138,Aluno de graduação,Instituto de Matemática e Estatística,True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus FEA,7:15:00 PM,True,8022,Rua do Matão 0,Ponto de ônibus metrô Butantã,10:40:00 PM,True,True
139,Aluno de pós-graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,8012,Ponto de ônibus - Acesso CPTM I,Ponto e ônibus História e Geografia,7:25:00 PM,True,8012,Ponto e ônibus História e Geografia,Ponto de ônibus - Acesso CPTM I,9:40:00 PM,True,True
142,Aluno de graduação,Escola Politécnica,True,"8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Metalúrgica,7:40:00 AM,True,"8012, 8032",Ponto de ônibus Poli Mecânica,Ponto de ônibus metrô Butantã,3:00:00 PM,True,True
143,Aluno de graduação,Escola Politécnica,True,"8012, 8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Biênio,8:30:00 AM,True,"8012, 8032",Ponto de ônibus Poli Biênio,Ponto de ônibus metrô Butantã,5:40:00 PM,True,True
145,Aluno de graduação,Instituto de Matemática e Estatística,True,8012,Ponto de ônibus metrô Butantã,Ponto de ônibus FEA,8:15:00 AM,True,8012,Ponto de ônibus Física,Ponto de ônibus metrô Butantã,11:50:00 AM,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4524,Aluno de graduação,"Faculdade de Filosofia, Letras e Ciências Humanas",True,"8012, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Biblioteca Brasiliana,8:00:00 AM,True,"8012, 8032",Ponto de ônibus CRUSP,Ponto de ônibus metrô Butantã,1:10:00 PM,True,True
4525,Aluno de graduação,Escola Politécnica,True,"8012, 8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Poli Civil,7:30:00 AM,True,"8012, 8022, 8032",Ponto de ônibus Poli Civil,Ponto de ônibus metrô Butantã,5:00:00 PM,True,True
4526,Aluno de graduação,Instituto de Biociências,True,8022,Ponto de ônibus metrô Butantã,Ponto de ônibus rua do Lago,7:00:00 AM,True,"8012, 8022",Ponto de ônibus Parada Matemática,Ponto de ônibus metrô Butantã,7:15:00 PM,True,True
4527,Aluno de graduação,Instituto de Biociências,True,"8022, 8032",Ponto de ônibus metrô Butantã,Ponto de ônibus Parada Matemática,7:40:00 AM,True,"8022, 8032",Ponto Paços das Artes,Ponto de ônibus metrô Butantã,10:15:00 PM,True,True


##### Calcular Porcentagem Carro por Instituto

In [659]:
porc_circular_por_instituto = pesquisa_vinculo_intervalo_terminal[['instituto','utiliza_circular_' + sentido]].groupby('instituto').agg('mean').reset_index()
porc_circular_por_instituto

,instituto,utiliza_circular_ida
0,Escola Politécnica,0.937500
1,Escola de Comunicações e Artes,0.921875
2,Escola de Educação Física e Esporte,0.937500
3,Faculdade de Arquitetura e Urbanismo,1.000000
4,Faculdade de Ciências Farmacêuticas,0.909091
5,"Faculdade de Economia, Administração, Contabil...",0.916667
6,Faculdade de Educação,0.812500
7,"Faculdade de Filosofia, Letras e Ciências Humanas",0.948864
8,Faculdade de Medicina Veterinária e Zootecnia,0.953488
9,"Faculdade de Medicina, Instituto de Ciências B...",0.876404


In [660]:
def porc_utiliza_circular(instituto):
    return porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido].iloc[0]


##### Calcula porcentagem para Ponto e Linha por Instituto (filtrando quem usa carro e terminal)

In [661]:
import random
def pessoas_por_linha(df_instituto_ponto):
    quant = {}
    for index, aluno in df_instituto_ponto.iterrows():
        str_linhas = aluno['linhas_' + sentido]
        linhas = str_linhas.split(', ')
        linha = random.choice(linhas)
        if linha not in quant:
            quant[linha] = 0
        quant[linha] += 1
        
        # Remove qualquer atividade do P3 com o 8032, que não passa por este terminal.
        if terminal == 'p3':
            if '8032' in quant:
                quant.pop('8032')
    
    return quant

def calcula_quantidade_pessoas_ponto(instituto, ponto):
    df_instituto = pesquisa_vinculo_intervalo_terminal[pesquisa_vinculo_intervalo_terminal['instituto'] == instituto]
    
    df_ponto = df_instituto[df_instituto['ponto_' + acao + '_' + sentido] == ponto]
    if len(df_ponto['ponto_' + acao + '_' + sentido]) == 0:
        return {}
    
    quant_pessoas_linha = pessoas_por_linha(df_ponto)

    return quant_pessoas_linha

In [662]:
pontos_existentes = pesquisa_vinculo_intervalo_terminal['ponto_' + acao + '_' + sentido].unique()

institutos_ponto_rel = {}
for instituto in pesquisa_vinculo_intervalo_terminal['instituto'].unique():
    institutos_ponto_rel[instituto] = {}
    soma = len(pesquisa_vinculo_intervalo_terminal[pesquisa_vinculo_intervalo_terminal['instituto'] == instituto]['instituto'])
    for ponto in pontos_existentes:
        rel_ponto_linhas = calcula_quantidade_pessoas_ponto(instituto, ponto)
        # Adiciona o ponto só se existe alguma linha que as pessoas pegam nele
        if len(rel_ponto_linhas) > 0:
            institutos_ponto_rel[instituto][ponto] = rel_ponto_linhas
            for linha in institutos_ponto_rel[instituto][ponto]:
                institutos_ponto_rel[instituto][ponto][linha] *= porc_utiliza_circular(instituto)
                institutos_ponto_rel[instituto][ponto][linha] *= porc_utiliza_terminal(instituto)
                institutos_ponto_rel[instituto][ponto][linha] /= soma
institutos_ponto_rel

{'Instituto de Matemática e Estatística': {'Ponto de ônibus FEA': {'8032': 0.27425772880318333,
   '8012': 0.31343740434649525,
   '8022': 0.10774410774410775},
  'Ponto de ônibus metrô Butantã': {'8032': 0.009794918885827977,
   '8012': 0.019589837771655953,
   '8022': 0.009794918885827977},
  'Ponto de ônibus CRUSP': {'8012': 0.009794918885827977},
  'Ponto de ônibus ECA': {'8032': 0.009794918885827977},
  'Ponto de ônibus FAU II': {'8022': 0.08815426997245178,
   '8012': 0.02938475665748393,
   '8032': 0.039179675543311906},
  'Ponto de ônibus Poli Eletrotécnica': {'8012': 0.009794918885827977,
   '8022': 0.009794918885827977},
  'Ponto de ônibus Praça do Relógio': {'8012': 0.009794918885827977,
   '8022': 0.009794918885827977},
  'Ponto de ônibus Física': {'8022': 0.009794918885827977,
   '8012': 0.009794918885827977}},
 'Faculdade de Filosofia, Letras e Ciências Humanas': {'Ponto de ônibus FEA': {'8022': 0.0026956353305785125},
  'Ponto e ônibus História e Geografia': {'8012': 0.0

# Pesquisa e (Aluno/DadosMatrusp ou Funcionário)

In [663]:
aulas_intervalo_horario = get_dados_vinculo(aulas_intervalo_horario, funcionarios_intervalo_horario)

##### Verificar unidades Matrusp e Pesquisa

In [664]:
# Alterar a unidade de aulas na qual a unidade não respondeu a pesquisa

for aula in aulas_intervalo_horario:
    if aula['unidade'] == 'Instituto de Ciências Biomédicas':
        aula['unidade'] = 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências'
    if aula['unidade'] == 'Instituto de Estudos Brasileiros':
        aula['unidade'] = 'Faculdade de Economia, Administração, Contabilidade e Atuária'
    if aula['unidade'] == 'Instituto de Psicologia':
        aula['unidade'] = 'Escola de Comunicações e Artes'

In [665]:
institutos_matrusp = {}
for aula in aulas_intervalo_horario:
    institutos_matrusp[aula['unidade']] = 1
institutos_matrusp = list(institutos_matrusp.keys())
institutos_matrusp.sort()
institutos_matrusp

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Escola de Educação Física e Esporte',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Pesquisas Energéticas e Nucleares',
 'Instituto de Química']

In [666]:
institutos_pesquisa = list(institutos_ponto_rel.keys())
institutos_pesquisa.sort()
institutos_pesquisa

['Escola Politécnica',
 'Escola de Comunicações e Artes',
 'Escola de Educação Física e Esporte',
 'Faculdade de Arquitetura e Urbanismo',
 'Faculdade de Ciências Farmacêuticas',
 'Faculdade de Economia, Administração, Contabilidade e Atuária',
 'Faculdade de Educação',
 'Faculdade de Filosofia, Letras e Ciências Humanas',
 'Faculdade de Medicina Veterinária e Zootecnia',
 'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências',
 'Faculdade de Odontologia',
 'Instituto Oceanográfico',
 'Instituto de Astronomia, Geofísica e Ciências Atmosféricas',
 'Instituto de Biociências',
 'Instituto de Ciências Biomédicas',
 'Instituto de Estudos Brasileiros',
 'Instituto de Física',
 'Instituto de Geociências',
 'Instituto de Matemática e Estatística',
 'Instituto de Pesquisas Energéticas e Nucleares',
 'Instituto de Psicologia',
 'Instituto de Química',
 'Instituto de Relações Internacionais',
 'Museu de Arte Contemporânea',
 'Reitoria Administra

In [667]:
for instituto_matrusp in institutos_matrusp:
    if instituto_matrusp not in institutos_pesquisa:
        print(instituto_matrusp + ' from matrusp is not in institutos pesquisa')

##### Calcula Map: dia->hora->ponto->linha->pessoas

In [668]:
dia_hora_ponto_linha = {}

def sum_valor_dia_hora_ponto_linha(dia, hora, ponto, linha, valor):
    if dia not in dia_hora_ponto_linha:
        dia_hora_ponto_linha[dia] = {}
    if hora not in dia_hora_ponto_linha[dia]:
        dia_hora_ponto_linha[dia][hora] = {}
    if ponto not in dia_hora_ponto_linha[dia][hora]:
        dia_hora_ponto_linha[dia][hora][ponto] = {}
    if linha not in dia_hora_ponto_linha[dia][hora][ponto]:
        dia_hora_ponto_linha[dia][hora][ponto][linha] = 0
    dia_hora_ponto_linha[dia][hora][ponto][linha] += valor

for aula in aulas_intervalo_horario:
    if aula['unidade'] in institutos_ponto_rel:
        porcentagem_pontos_por_instituto = institutos_ponto_rel[aula['unidade']]
        for ponto in porcentagem_pontos_por_instituto:
            linhas = porcentagem_pontos_por_instituto[ponto]
            for linha in linhas:
                porc = linhas[linha]
                valor = aula['total'] * porc
                sum_valor_dia_hora_ponto_linha(aula['dia'], hora, ponto, linha, valor)

# arredondando
for dia in dia_hora_ponto_linha:
    for hora in dia_hora_ponto_linha[dia]:
        for ponto in dia_hora_ponto_linha[dia][hora]:
            for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                dia_hora_ponto_linha[dia][hora][ponto][linha] = round(dia_hora_ponto_linha[dia][hora][ponto][linha])

dia_hora_ponto_linha

{'sex': {480: {'Ponto de ônibus FEA': {'8032': 437, '8012': 509, '8022': 205},
   'Ponto de ônibus metrô Butantã': {'8032': 62, '8012': 106, '8022': 80},
   'Ponto de ônibus CRUSP': {'8012': 62, '8032': 38, '8022': 71},
   'Ponto de ônibus ECA': {'8032': 16, '8012': 26, '8022': 11},
   'Ponto de ônibus FAU II': {'8022': 128, '8012': 79, '8032': 85},
   'Ponto de ônibus Poli Eletrotécnica': {'8012': 67, '8022': 61, '8032': 29},
   'Ponto de ônibus Praça do Relógio': {'8012': 14, '8022': 8, '8032': 4},
   'Ponto de ônibus Física': {'8022': 40, '8012': 46, '8032': 11},
   'Ponto de ônibus Biblioteca Brasiliana': {'8012': 243,
    '8032': 192,
    '8022': 51},
   'Ponto de ônibus Portaria III': {'8022': 328, '8012': 312},
   'Ponto de ônibus Biblioteca Farmácia e Química': {'8022': 277,
    '8012': 31,
    '8032': 11},
   'Ponto Clube dos funcionários': {'8022': 30, '8012': 32, '8032': 5},
   'Ponto de ônibus  Parada Matemática': {'8012': 23, '8022': 33, '8032': 5},
   'Ponto de ônibus CEP

##### Salva Map em Arquivo

In [669]:
import numpy as np

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

def encode(s):
    return s.encode('latin1').decode('unicode_escape')

with open(map_file_name, 'w+') as f:
    str_dia_hora_ponto_linha = encode(json.dumps(dia_hora_ponto_linha, indent=4, cls=NpEncoder))
    f.write(str_dia_hora_ponto_linha)
    print(str_dia_hora_ponto_linha)

{
    "sex": {
        "480": {
            "Ponto de ônibus FEA": {
                "8032": 437,
                "8012": 509,
                "8022": 205
            },
            "Ponto de ônibus metrô Butantã": {
                "8032": 62,
                "8012": 106,
                "8022": 80
            },
            "Ponto de ônibus CRUSP": {
                "8012": 62,
                "8032": 38,
                "8022": 71
            },
            "Ponto de ônibus ECA": {
                "8032": 16,
                "8012": 26,
                "8022": 11
            },
            "Ponto de ônibus FAU II": {
                "8022": 128,
                "8012": 79,
                "8032": 85
            },
            "Ponto de ônibus Poli Eletrotécnica": {
                "8012": 67,
                "8022": 61,
                "8032": 29
            },
            "Ponto de ônibus Praça do Relógio": {
                "8012": 14,
                "8022": 8,
                "8

In [670]:
# Verificação de dia_hora_ponto_linha em relação ao total do MATRUSP (só funciona para alunos)
if vinculo == 'aluno':
    sum_total = 0
    for dia in dia_hora_ponto_linha:
        for hora in dia_hora_ponto_linha[dia]:
            for ponto in dia_hora_ponto_linha[dia][hora]:
                for linha in dia_hora_ponto_linha[dia][hora][ponto]:
                    sum_total += dia_hora_ponto_linha[dia][hora][ponto][linha]
    print(sum_total)

    sum_aulas = 0
    for aula in aulas_intervalo_horario:
        sum_aulas += aula['total']
    media_utiliza_circular = porc_circular_por_instituto['utiliza_circular_' + sentido].mean()
    print(round(sum_aulas * media_utiliza_circular))

39995
40711


## Add employees and professors
TODO: considerar funcionários e professores estrapolando o horário de entrada da pesquisa

##### Número Pessoas por Dia e Instituto

In [671]:
agrupamento_dia_instituto = {}
for dia in dias:
    agrupamento_dia_instituto[dia] = {}
for aula in aulas_intervalo_horario:
    if aula['unidade'] not in agrupamento_dia_instituto[aula['dia']]:
        agrupamento_dia_instituto[aula['dia']][aula['unidade']] = 0
    agrupamento_dia_instituto[aula['dia']][aula['unidade']] += aula['total']
agrupamento_dia_instituto

{'seg': {'Instituto de Geociências': 100,
  'Faculdade de Medicina Veterinária e Zootecnia': 558,
  'Escola de Comunicações e Artes': 612,
  'Escola Politécnica': 2096,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 1093,
  'Instituto de Matemática e Estatística': 1118,
  'Faculdade de Arquitetura e Urbanismo': 314,
  'Faculdade de Educação': 16,
  'Faculdade de Ciências Farmacêuticas': 157,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 452,
  'Escola de Educação Física e Esporte': 65,
  'Faculdade de Odontologia': 290,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 313,
  'Instituto de Pesquisas Energéticas e Nucleares': 42,
  'Instituto de Física': 345,
  'Instituto de Química': 157,
  'Instituto Oceanográfico': 63,
  'Instituto de Biociências': 36,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 5},
 'ter': {'Instituto de Química': 401,
  'Escola de Comunicações e Artes': 686,

In [672]:
for dia in agrupamento_dia_instituto:
    for instituto in agrupamento_dia_instituto[dia]:
        if len(porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido]) > 0:
            agrupamento_dia_instituto[dia][instituto] *= porc_circular_por_instituto[porc_circular_por_instituto['instituto'] == instituto]['utiliza_circular_' + sentido].iloc[0]
        else:
            agrupamento_dia_instituto[dia][instituto] = 0
        agrupamento_dia_instituto[dia][instituto] = int(agrupamento_dia_instituto[dia][instituto])

# Pessoas por instituto que utilizam circular de acordo com a pesquisa
agrupamento_dia_instituto

{'seg': {'Instituto de Geociências': 100,
  'Faculdade de Medicina Veterinária e Zootecnia': 532,
  'Escola de Comunicações e Artes': 564,
  'Escola Politécnica': 1965,
  'Faculdade de Filosofia, Letras e Ciências Humanas': 1037,
  'Instituto de Matemática e Estatística': 1084,
  'Faculdade de Arquitetura e Urbanismo': 314,
  'Faculdade de Educação': 13,
  'Faculdade de Ciências Farmacêuticas': 142,
  'Faculdade de Economia, Administração, Contabilidade e Atuária': 414,
  'Escola de Educação Física e Esporte': 60,
  'Faculdade de Odontologia': 290,
  'Faculdade de Medicina, Instituto de Ciências Biomédicas, Instituto de Química e Instituto de Biociências': 274,
  'Instituto de Pesquisas Energéticas e Nucleares': 42,
  'Instituto de Física': 333,
  'Instituto de Química': 153,
  'Instituto Oceanográfico': 63,
  'Instituto de Biociências': 34,
  'Instituto de Astronomia, Geofísica e Ciências Atmosféricas': 5},
 'ter': {'Instituto de Química': 392,
  'Escola de Comunicações e Artes': 632,